In [20]:
import  numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

# 1. Carregando data.csv
data = pd.read_csv('Data_CSV/data_alzheimer.csv')

# Breast_cancer_data_Suwal_2018.csv
# Data_Alzheimer_Diagnosis.csv
# data_Breast_Cancer_Wisconsin.csv
# data_Mammography.csv

from sklearn.impute import SimpleImputer

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Fit and transform the data
data_filled = imputer.fit_transform(data)
data_filled = pd.DataFrame(data_filled, columns=data.columns)

data = data_filled

# Supondo que a última coluna seja a classe
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


print(X.shape)
print(y.shape)

# Converta para DataFrame para facilitar a seleção de características
# df = pd.DataFrame(X)
# corr_matrix = df.corr().abs()

# Encontre características altamente correlacionadas
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]  # Limite

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Remova as mesmas características de ambos os conjuntos (treinamento e teste)
# X_train_reduced = X_train.drop(to_drop, axis=1)
# X_test_reduced = X_test.drop(to_drop, axis=1)

print(f"Shape of X_train_reduced: {X_train_reduced.shape}")
print(f"Shape of X_test_reduced: {X_test_reduced.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

(820, 29)
(820,)
Shape of X_train_reduced: (738, 29)
Shape of X_test_reduced: (82, 29)
Shape of y_train: (738,)
Shape of y_test: (82,)


In [21]:
from sklearn.preprocessing import StandardScaler  # Adicione essa linha para importar StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA


# # Normalizar os dados
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# #PCA analysis
# pca = PCA(n_components=0.95)  # Preserve 95% of the variance
# X_train_scaled = pca.fit_transform(X_train_scaled)
# X_test_scaled = pca.transform(X_test_scaled)

# Inicializar o Bayesiano
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_scaled, y_train)
nb_predictions = nb_classifier.predict(X_test_scaled)

# Inicializar o LDA com shrinkage para reduzir a colinearidade
lda_classifier = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
lda_classifier.fit(X_train_scaled, y_train)
lda_predictions = lda_classifier.predict(X_test_scaled)

# Quadrado
qda_classifier = QuadraticDiscriminantAnalysis(reg_param=0.1)
qda_classifier.fit(X_train_scaled, y_train)
qda_predictions = qda_classifier.predict(X_test_scaled)

In [22]:
# 4. Avaliação dos modelos
#Função para avaliar cada modelo

def evaluate_with_confusion_matrix(name, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        print(f'\n{name} Classifier - Confusion Matrix:')
        print(cm)

    # Acurácia por classe
        class_accuracies = cm.diagonal() / cm.sum(axis = 1)
        print(f'Acurácia por classe:{class_accuracies}')

    # Acurácia geral
        overall_accuracy = cm.diagonal().sum() / cm.sum()
        print(f'Acurácia geral: {overall_accuracy:.4f}')
        print(classification_report(y_true, y_pred))

evaluate_with_confusion_matrix('Naive Bayes',y_test, nb_predictions)
evaluate_with_confusion_matrix('Linear Discriminant Analysis', y_test, lda_predictions)
evaluate_with_confusion_matrix('Quadratic Discriminant Analysis', y_test, qda_predictions)


Naive Bayes Classifier - Confusion Matrix:
[[20  7]
 [10 45]]
Acurácia por classe:[0.74074074 0.81818182]
Acurácia geral: 0.7927
              precision    recall  f1-score   support

         0.0       0.67      0.74      0.70        27
         1.0       0.87      0.82      0.84        55

    accuracy                           0.79        82
   macro avg       0.77      0.78      0.77        82
weighted avg       0.80      0.79      0.80        82


Linear Discriminant Analysis Classifier - Confusion Matrix:
[[17 10]
 [ 2 53]]
Acurácia por classe:[0.62962963 0.96363636]
Acurácia geral: 0.8537
              precision    recall  f1-score   support

         0.0       0.89      0.63      0.74        27
         1.0       0.84      0.96      0.90        55

    accuracy                           0.85        82
   macro avg       0.87      0.80      0.82        82
weighted avg       0.86      0.85      0.85        82


Quadratic Discriminant Analysis Classifier - Confusion Matrix:
[[19 

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carregar os dados do arquivo CSV
df = data

# Supondo que a última coluna seja a variável alvo (classe)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Converter rótulos para valores numéricos, se necessário
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Criar o modelo Discriminante Linear
model = LinearDiscriminantAnalysis()
print('\n\n\n\n Linear')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 100  # Número de iterações
test_size = 0.2  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())

model = LinearDiscriminantAnalysis()
print('\n\n\n\n Quadrático')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 100  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())



model = GaussianNB()
print('\n\n\n\n Naive Bayes')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 100  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())







 Linear
Acurácia por repetição: [0.93902439 0.92682927 0.95731707 0.95731707 0.9695122  0.93902439
 0.96341463 0.93902439 0.9695122  0.95121951 0.91463415 0.94512195
 0.93902439 0.95731707 0.94512195 0.92682927 0.93902439 0.92682927
 0.94512195 0.94512195 0.93902439 0.98170732 0.93902439 0.94512195
 0.9695122  0.98780488 0.95731707 0.95121951 0.92073171 0.96341463
 0.93902439 0.94512195 0.95121951 0.93902439 0.95731707 0.93902439
 0.95731707 0.95121951 0.90853659 0.95731707 0.96341463 0.95121951
 0.94512195 0.90853659 0.95731707 0.95121951 0.93902439 0.94512195
 0.94512195 0.95731707 0.90853659 0.94512195 0.93902439 0.90853659
 0.95731707 0.9695122  0.96341463 0.94512195 0.95731707 0.94512195
 0.93902439 0.95731707 0.92682927 0.95121951 0.93902439 0.95121951
 0.92682927 0.91463415 0.93902439 0.93292683 0.95121951 0.92073171
 0.95731707 0.96341463 0.94512195 0.93292683 0.95731707 0.92073171
 0.94512195 0.93902439 0.93902439 0.95731707 0.93902439 0.95121951
 0.93292683 0.95731707 0.9

In [24]:
import numpy as np

# Generating non-uniform data
original_data = np.random.exponential(scale=50, size=256)

# Splitting data into 8 bins
split_data = np.array_split(original_data, 8)

# Calculating the mean of original data
original_mean = np.mean(original_data)

# Calculating the mean of each new bin and overall mean
new_means = [np.mean(bin) for bin in split_data]
overall_new_mean = np.mean(new_means)

print(f"Original Mean: {original_mean}")
print(f"Means of New Bins: {new_means}")
print(f"Overall New Mean: {overall_new_mean}")


Original Mean: 50.17097903302651
Means of New Bins: [np.float64(53.05168565353928), np.float64(67.05060892671652), np.float64(52.76203603529248), np.float64(34.12083593020547), np.float64(38.784432125144264), np.float64(46.14795321402105), np.float64(37.69138521756676), np.float64(71.75889516172624)]
Overall New Mean: 50.17097903302651
